In [19]:
import os
import random
import tensorflow as tf
import keras
from keras import backend
from keras.utils import get_file
from keras.utils import get_source_inputs
from tensorflow.keras.utils import unpack_x_y_sample_weight
from keras import layers
import numpy as np
from matplotlib import cm

In [32]:
left_dir = "D:/datasets/kitti_2015_and_2012/training/left"

os.listdir(left_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/datasets/kitti_2015_and_2012/training/left'

In [44]:
os.path.exists("/home/ubuntu24/datasets/kitti_2015_and_2012/training/left")

True

In [26]:
def bilinear_sampler(imgs, coords):
    """
    Construct a new image by bilinear sampling from the input image.
    Points falling outside the source image boundary have value 0.
    Args:
        imgs: source image to be sampled from [batch, height_s, width_s, channels]
        coords: coordinates of source pixels to sample from [batch, height_t,width_t, 2].
                height_t/width_t correspond to the dimensions of the output image
                (don't need to be the same as height_s/width_s). The two channels
                correspond to x and y coordinates respectively.
    Returns:
        A new sampled image [batch, height_t, width_t, channels]
    """

    def _repeat(x, n_repeats):
        rep = keras.ops.transpose(
            keras.ops.expand_dims(keras.ops.ones(shape=keras.ops.stack([
                n_repeats,
            ])), 1), [1, 0])
        rep = keras.ops.cast(rep, 'float32')
        x = keras.ops.matmul(keras.ops.reshape(x, (-1, 1)), rep)
        return keras.ops.reshape(x, [-1])

    coords_x, coords_y = keras.ops.split(coords, 2, axis=3)
    inp_size = keras.ops.shape(imgs)
    coord_size = keras.ops.shape(coords)
    out_size = [coord_size[0], coord_size[1], coord_size[2], inp_size[3]]

    coords_x = keras.ops.cast(coords_x, 'float32')
    coords_y = keras.ops.cast(coords_y, 'float32')

    x0 = keras.ops.floor(coords_x)
    x1 = x0 + 1
    y0 = keras.ops.floor(coords_y)
    y1 = y0 + 1

    y_max = keras.ops.cast(inp_size[1] - 1, 'float32')
    x_max = keras.ops.cast(inp_size[2] - 1, 'float32')
    zero = keras.ops.zeros([1], dtype='float32')

    wt_x0 = x1 - coords_x
    wt_x1 = coords_x - x0
    wt_y0 = y1 - coords_y
    wt_y1 = coords_y - y0

    x0_safe = keras.ops.clip(x0, zero[0], x_max)
    y0_safe = keras.ops.clip(y0, zero[0], y_max)
    x1_safe = keras.ops.clip(x1, zero[0], x_max)
    y1_safe = keras.ops.clip(y1, zero[0], y_max)

    ## indices in the flat image to sample from
    dim2 = keras.ops.cast(inp_size[2], 'float32')
    dim1 = keras.ops.cast(inp_size[2] * inp_size[1], 'float32')
    base = keras.ops.reshape(
        _repeat(keras.ops.cast(keras.ops.arange(coord_size[0]), 'float32') * dim1, coord_size[1] * coord_size[2]),
        [out_size[0], out_size[1], out_size[2], 1]
    )

    base_y0 = base + y0_safe * dim2
    base_y1 = base + y1_safe * dim2
    idx00 = x0_safe + base_y0
    idx01 = x0_safe + base_y1
    idx10 = x1_safe + base_y0
    idx11 = x1_safe + base_y1

    ## sample from imgs
    im00 = keras.ops.take(imgs, keras.ops.cast(idx00, 'int32'))
    im01 = keras.ops.take(imgs, keras.ops.cast(idx01, 'int32'))
    im10 = keras.ops.take(imgs, keras.ops.cast(idx10, 'int32'))
    im11 = keras.ops.take(imgs, keras.ops.cast(idx11, 'int32'))

    w00 = wt_x0 * wt_y0
    w01 = wt_x0 * wt_y1
    w10 = wt_x1 * wt_y0
    w11 = wt_x1 * wt_y1

    output = keras.ops.add(
        keras.ops.add(w00 * im00, w01 * im01),
        keras.ops.add(w10 * im10, w11 * im11)
    )

    return output

In [14]:
class BilinearSampler(keras.layers.Layer):

    def __init__(self):
        super(BilinearSampler, self).__init__()

    def _repeat(self, x, n_repeats):
        rep = keras.ops.transpose(
            keras.ops.expand_dims(keras.ops.ones(shape=keras.ops.stack([
                n_repeats,
            ])), 1), [1, 0])
        rep = keras.ops.cast(rep, 'float32')
        x = keras.ops.matmul(keras.ops.reshape(x, (-1, 1)), rep)
        return keras.ops.reshape(x, [-1])

    def call(self, imgs, coords):
        coords_x, coords_y = keras.ops.split(coords, 2, axis=3)
        inp_size = keras.ops.shape(imgs)
        coord_size = keras.ops.shape(coords)
        out_size = [coord_size[0], coord_size[1], coord_size[2], inp_size[3]]

        coords_x = keras.ops.cast(coords_x, 'float32')
        coords_y = keras.ops.cast(coords_y, 'float32')

        x0 = keras.ops.floor(coords_x)
        x1 = x0 + 1
        y0 = keras.ops.floor(coords_y)
        y1 = y0 + 1

        y_max = keras.ops.cast(inp_size[1] - 1, 'float32')
        x_max = keras.ops.cast(inp_size[2] - 1, 'float32')
        zero = keras.ops.zeros([1], dtype='float32')

        wt_x0 = x1 - coords_x
        wt_x1 = coords_x - x0
        wt_y0 = y1 - coords_y
        wt_y1 = coords_y - y0

        x0_safe = keras.ops.clip(x0, zero[0], x_max)
        y0_safe = keras.ops.clip(y0, zero[0], y_max)
        x1_safe = keras.ops.clip(x1, zero[0], x_max)
        y1_safe = keras.ops.clip(y1, zero[0], y_max)

        ## indices in the flat image to sample from
        dim2 = keras.ops.cast(inp_size[2], 'float32')
        dim1 = keras.ops.cast(inp_size[2] * inp_size[1], 'float32')
        base = keras.ops.reshape(
            self._repeat(keras.ops.cast(keras.ops.arange(coord_size[0]), 'float32') * dim1, coord_size[1] * coord_size[2]),
            [out_size[0], out_size[1], out_size[2], 1]
        )

        base_y0 = base + y0_safe * dim2
        base_y1 = base + y1_safe * dim2
        idx00 = x0_safe + base_y0
        idx01 = x0_safe + base_y1
        idx10 = x1_safe + base_y0
        idx11 = x1_safe + base_y1

        ## sample from imgs
        im00 = keras.ops.take(imgs, keras.ops.cast(idx00, 'int32'))
        im01 = keras.ops.take(imgs, keras.ops.cast(idx01, 'int32'))
        im10 = keras.ops.take(imgs, keras.ops.cast(idx10, 'int32'))
        im11 = keras.ops.take(imgs, keras.ops.cast(idx11, 'int32'))

        w00 = wt_x0 * wt_y0
        w01 = wt_x0 * wt_y1
        w10 = wt_x1 * wt_y0
        w11 = wt_x1 * wt_y1

        output = keras.ops.add(
            keras.ops.add(w00 * im00, w01 * im01),
            keras.ops.add(w10 * im10, w11 * im11)
        )

        return output

In [15]:
bilinear_sampler = BilinearSampler()

In [16]:
batch_size = 1
imgs = keras.ops.ones((batch_size, 15, 20, 128))
coords = keras.ops.ones((batch_size, 15, 20, 2))

In [17]:
bilinear_out = bilinear_sampler(imgs, coords)